In [1]:
import pip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
song_app = pd.read_csv("../../SongApp.csv",header=-1,)


In [3]:
song_app.head()
CUDA_VISIBLE_DEVICES=""

In [4]:
song_app.size

42661872

In [5]:
song_app.shape

(463716, 92)

In [6]:
# clean data
df_song = song_app.dropna()
df_song.__delitem__(0)

In [7]:
df_song.head()

,1,2,3,4,5,6,7,8,9,10,...,82,83,84,85,86,87,88,89,90,91
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
5,2001,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


In [8]:
labels_column = df_song.iloc[0:,0]
labels = df_song.values[0:,0]
labels

array([ 2001.,  2001.,  2001., ...,  2006.,  2007.,  2007.])

In [9]:
df_song.__delitem__(1)

In [10]:
df_song['labels'] = labels_column

/home/phoenix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
num_columns = df_song.columns.shape[0]
columns = list(range(num_columns - 1))
columns_name = list(map(str,columns)) + ['labels']
df_song.columns = columns_name
df_song.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,labels
1,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,...,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061,2001
2,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,...,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345,2001
3,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,...,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382,2001
4,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,...,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903,2001
5,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,-4.69088,...,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263,2001


In [13]:
x_data = df_song.sample(50000)
y_data = x_data.values[0:,90]
y_data

array([ 2006.,  2005.,  2009., ...,  1995.,  2006.,  1990.])

In [14]:
x_data.__delitem__('labels')
x_data.head()

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
114791,45.59051,42.06787,39.72611,27.87376,6.73239,7.91988,-11.83957,3.47484,13.51811,8.21595,...,33.09377,-109.23271,90.26132,177.36077,15.52546,46.54546,215.14015,-1.47362,-51.77191,0.88874
170446,41.21929,7.48728,-13.87841,3.93196,7.50054,-18.06949,7.06437,3.36844,2.91055,4.55294,...,-35.38574,-50.22039,18.25810,29.99592,-8.13865,15.47705,344.90017,7.86129,21.36647,1.12312
337747,51.23714,48.53738,57.53652,28.08185,-12.98128,-1.30023,6.63941,1.97967,1.85686,10.33589,...,-10.10465,-65.92688,104.77308,78.77191,-13.46322,72.78617,-71.77182,7.58393,179.23668,-4.45921
49719,41.60582,117.21378,-10.72843,-16.63562,-28.32313,-18.29677,-41.34251,-5.44953,14.50211,13.16637,...,-5.62751,-27.14093,1.28082,91.21903,2.83053,14.45348,-9.08064,5.76369,-38.50147,11.44578
101611,39.47102,-55.66652,-15.79422,-7.91305,16.39545,8.73369,10.16690,-1.99703,2.34454,-1.35384,...,6.48487,40.88504,50.71887,267.79105,2.50387,56.42598,81.58127,22.15831,194.95616,16.94858


In [15]:
# split into training and test data
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(45000, 90)
(5000, 90)
(45000,)
(5000,)


In [16]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(y_train.shape[0], input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
X_train.shape

(45000, 90)

In [17]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=2, batch_size=2, verbose=0)

In [18]:
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -366200.21 (223817.43) MSE
